<img src="Imagenes/bannermodelado_cnn.jpg" alt="Imagen creada con inteligencia artificial y editada con Microsoft Paint" style="border-radius: 15px; width: 95%;">

*Imagen creada con inteligencia artificial*


## **BIBLIOTECAS USADAS:**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report 
import os
from keras.models import save_model 
from tensorflow.keras.applications import VGG16  
from tensorflow.keras import layers, models, optimizers

## **CARGA DEL CONJUNTO DE DATOS 'FER-2013'** 
>**Realmente no cargamos el dataset 'fer2013' tal cual; cargamos un dataset obtenido tras aplicar técnicas de data augmentation. En el Jupyter Notebook 'Data_augmentation_fer2013' se realiza y explica el proceso.**

In [2]:
df = pd.read_csv('datos/fer2013/fer2013_blc_todos_rotados.csv')
print("El dataset ha sido cargado correctamente.") 
df.head(5)

El dataset ha sido cargado correctamente.


,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


## **PREPARACIÓN PREVIA AL SPLIT**  
>>**La columna 'Usage' no nos es útil. Le ponemos cara de asco y la borramos.**

In [3]:
df.drop(columns=['Usage'], inplace=True)

## **SEPARAMOS "X" E "y" PIXELES Y ETIQUETAS.** 
> **La columna 'emotion' contiene las etiquetas, y la columna 'pixels' contiene las cadenas de números que dan lugar a las fotografías.**

In [4]:
X = df.drop(columns=['emotion'])  
y = df['emotion']  

## **CONVERSIÓN A ARRAY Y NORMALIZACIÓN DE LA COLUMNA 'PIXELS'**  
>**Los modelos, al menos con los que trabajaremos, necesitan arrays de NumPy; no pueden trabajar con cadenas. Además, deben estar normalizados.**

In [5]:
def string_to_image_array(string):
    pixels = np.array(string.split(), dtype=np.float32)
    return pixels.reshape((48, 48, 1))

X_pixels = np.array([string_to_image_array(pixels) for pixels in X['pixels']])
X_pixels = X_pixels.astype('float32')
X_pixels /= 255.0 

print("Shape del array X_train_pixels:", X_pixels.shape)

Shape del array X_train_pixels: (125846, 48, 48, 1)


## **SPLIT**  
>**Dividiremos nuestro dataset en tres partes: train, validación y test.**

In [6]:
X_train, X_temp, y_train, y_temp = train_test_split(X_pixels, y, test_size=0.2, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)
print("¿Se han separado de forma balanceada?")
print("Tanto por 1 de clases en y_train:")
print(y_train.value_counts(True))

print("\nTanto por 1 de clases en y_val:")
print(y_val.value_counts(True))

print("\nTanto por 1 de clases en y_test:")
print(y_test.value_counts(True))

¿Se han separado de forma balanceada?
Tanto por 1 de clases en y_train:
emotion
1    0.142864
0    0.142864
3    0.142854
6    0.142854
4    0.142854
5    0.142854
2    0.142854
Name: proportion, dtype: float64

Tanto por 1 de clases en y_val:
emotion
2    0.142868
6    0.142868
5    0.142868
3    0.142868
0    0.142868
4    0.142868
1    0.142789
Name: proportion, dtype: float64

Tanto por 1 de clases en y_test:
emotion
1    0.142868
2    0.142868
3    0.142868
4    0.142868
5    0.142868
6    0.142868
0    0.142789
Name: proportion, dtype: float64


>Sí, la estratificación ha sido correcta. El porcentaje de cada clase es muy similar en train, validación y test.

# **MODELOS:**  
>**En primer lugar, definiremos, entrenaremos y evaluaremos un modelo de Red Neuronal Convolucional (CNN). En segundo lugar, haremos lo mismo con un modelo de Transfer Learning basado en VGG16.**

## **DEFINICIÓN DEL MODELO CNN**

In [7]:
model_cnn = Sequential()

# Primera capa convolucional
model_cnn.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1))) #El modelo trabaja con arrays de 48x48 de una sola dimension, o sea capas... En blanco y negro, hablando en plata.
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

# Segunda capa convolucional
model_cnn.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

# Tercera capa convolucional
model_cnn.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

# Cuarta capa convolucional
model_cnn.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

# Capa de flatten para conectar con la capa densa
model_cnn.add(Flatten())

# Quinta capa densa (totalmente conectada)
model_cnn.add(Dense(256, activation='relu'))
model_cnn.add(Dropout(0.5))  # Dropout para reducir overfitting

# Sexta capa densa (totalmente conectada)
model_cnn.add(Dense(128, activation='relu'))
model_cnn.add(Dropout(0.5))  # Dropout para reducir overfitting

# Capa de salida (no se cambia)
model_cnn.add(Dense(7, activation='softmax'))  # Capa de salida con 7 clases (expresiones faciales)


C:\Users\pepea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## **COMPILADO DEL MODELO CNN**

In [8]:
model_cnn.compile(loss='sparse_categorical_crossentropy',  
              optimizer=Adam(learning_rate=0.0002),  #He ido probando, este ha dado muy buen resultado
              metrics=['accuracy'])  # Buscamos la mayor presicion

## **CALLBACK PARA EL MODELO CNN**   
>**Para detener el entrenamiento si la mejora en la métrica se detiene.**

In [9]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=40, restore_best_weights=True) 

>**El hiperparámetro 'patience' se llama así por algo, es decir... si aumentas 'patience', el modelo tendrá más paciencia, seguirá durante más épocas sin ver mejoras (a la espera de que mejore en otra o se acaben las épocas).**

## **ENTRENAMIENTO DEL MODELO CNN**

In [10]:
history = model_cnn.fit(X_train, y_train,
                    batch_size=70,  #Se han probado diferentes, este parece que tiene buena relacion tiempo/calidad
                    epochs=115,  
                    verbose=1,
                    validation_data=(X_val, y_val),  
                    callbacks=[early_stopping])

Epoch 1/115
1439/1439 ━━━━━━━━━━━━━━━━━━━━ 192s 132ms/step - accuracy: 0.2086 - loss: 1.8715 - val_accuracy: 0.4545 - val_loss: 1.4219
Epoch 2/115
1439/1439 ━━━━━━━━━━━━━━━━━━━━ 194s 135ms/step - accuracy: 0.4758 - loss: 1.3990 - val_accuracy: 0.5593 - val_loss: 1.1799
Epoch 3/115
1439/1439 ━━━━━━━━━━━━━━━━━━━━ 188s 130ms/step - accuracy: 0.5653 - loss: 1.1676 - val_accuracy: 0.6124 - val_loss: 1.0382
Epoch 4/115
1439/1439 ━━━━━━━━━━━━━━━━━━━━ 184s 128ms/step - accuracy: 0.6221 - loss: 1.0113 - val_accuracy: 0.6411 - val_loss: 0.9413
Epoch 5/115
1439/1439 ━━━━━━━━━━━━━━━━━━━━ 182s 126ms/step - accuracy: 0.6646 - loss: 0.9029 - val_accuracy: 0.6610 - val_loss: 0.8988
Epoch 6/115
1439/1439 ━━━━━━━━━━━━━━━━━━━━ 182s 126ms/step - accuracy: 0.6956 - loss: 0.8176 - val_accuracy: 0.6718 - val_loss: 0.8646
Epoch 7/115
1439/1439 ━━━━━━━━━━━━━━━━━━━━ 183s 127ms/step - accuracy: 0.7317 - loss: 0.7245 - val_accuracy: 0.6888 - val_loss: 0.8246
Epoch 8/115
1439/1439 ━━━━━━━━━━━━━━━━━━━━ 181s 126ms/s

6 horas y 8 minutos de entrenamiento.  
>Obviamente dependerá del ordenador.

## **EVALUACION CONTRA EL TEST** 
>**El modelo, en su entrenamiento, no ha visto estos ni de refilon**

In [11]:
loss, accuracy = model_cnn.evaluate(X_test, y_test, verbose=0)
print(f'Loss en el conjunto de prueba:: {loss:.4f}')
print(f'Accuracy en el conjunto de prueba:: {accuracy*100:.2f}%')

Loss en el conjunto de prueba:: 1.5009
Accuracy en el conjunto de prueba:: 81.17%


Loss en el conjunto de prueba: 1.5009  
Accuracy en el conjunto de prueba:: 81.17%

> **En un principio, una precisión del 81.17% puede parecer relativamente baja, pero es un resultado aceptable para el reconocimiento de expresiones faciales, especialmente considerando el conjunto de datos FER2013 utilizado. El Ministerio de Bienestar Emocional debería estar satisfecho con este desempeño inicial.**

## **PREDICCIONES CNN**

In [12]:
y_pred = model_cnn.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

394/394 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step


## **CLASSIFICATION REPORT CNN**

In [13]:
class_names = ['Enfado', 'Asco', 'Miedo', 'Felicidad', 'Tristeza', 'Sorpresa', 'Neutral']
print(classification_report(y_test, y_pred_classes, target_names=class_names))

              precision    recall  f1-score   support

      Enfado       0.80      0.73      0.76      1797
        Asco       0.98      0.99      0.98      1798
       Miedo       0.71      0.71      0.71      1798
   Felicidad       0.86      0.88      0.87      1798
    Tristeza       0.70      0.71      0.71      1798
    Sorpresa       0.90      0.91      0.90      1798
     Neutral       0.73      0.76      0.74      1798

    accuracy                           0.81     12585
   macro avg       0.81      0.81      0.81     12585
weighted avg       0.81      0.81      0.81     12585



## **GUARDADO DEL MODELO CNN**

In [14]:

save_dir = 'modelos_entrenados'

model_cnn.save(os.path.join(save_dir, 'modelo_cnn.h5'))

save_model(model_cnn, os.path.join(save_dir, 'modelo_cnn.keras'))



> **El modelo entrenado ha sido guardado en dos formatos diferentes. Aunque ambos, el formato h5 y el formato keras, mantienen la misma calidad de los datos, la elección del formato puede afectar su usabilidad a largo plazo. Por ello, hemos optado por ofrecer ambas opciones al cliente para que pueda seleccionar la más conveniente según sus necesidades futuras.**

## **RESUMEN/EXPLICACIÓN DEL MODELO CNN**

Este modelo CNN (Convolutional Neural Network) está diseñado para reconocer las expresiones faciales en fotografias en blanco y negro de 48x48 píxeles, segun 
chat gtp son algo así como un cerebro digital que aprende a identificar emociones (realmente expresiones faciales) en fotos. 🧠

- **Capas Convolucionales**: Son como filtros que detectan características como bordes y texturas en las imágenes  
propias de cada expresion, es decir busca las caracteriasticas comunes a cada una de las 7 expresiones faciales 
que este modelo reconoce, otorgandole una etiqueta a cada una.    

- **Capas Densas**: Son como capas finales que interpretan estas características para decidir qué emoción se ve en la imagen.    
Digamos que es como un inspector de policia que ve las pruebas (las caracteristicas) y dice "El sujeto está feliz"  
  
- **Dropout**: Es una técnica para evitar que el modelo memorice demasiado y pueda generalizar mejor.    
¿Conoces esa fotografia de un hombre durmiendo en un colchón que tiene su forma? pues con dropout se busca evitar eso.  
  
- **Función de Pérdida**: Durante el entrenamiento (usando la parte de datos reservada para validación) comprueba como   
de grande son los fallos que está teniendo.    

- **Compilación**:  Es como organizar la granja por la mañana. Preparamos al modelo diciéndole qué errores evitar,    
cómo ajustar los pesos de las neuronas (con Adam, ahora lo vemos), y qué métricas usar para ver como de bien está aprendiendo.  
  
- **Optimizador (Adam)**:  El optimizador Adam ajusta cómo la CNN aprende de los datos. Es como un "entrenador personal" (O el jefe de la granja)    
que guía al modelo sobre cómo mejorar durante el entrenamiento, ajustando los pesos y sesgos de las neuronas para que pueda    
predecir mejor las expresiones faciales en las imágenes.    
  
- **Callback (EarlyStopping)**: Es como "el capataz" durante el entrenamiento. Monitorea como la precisión en los datos de validación    
va mejorando y detiene el entrenamiento cuando la precisión deja de mejorar, restaurando los mejores pesos del modelo.    
Esto ayuda a prevenir el sobreajuste y asegura que el modelo generalize bien a nuevos datos.    

- **Métricas**: Digamos que son como las notas del cole, debemos saber como de bueno es el modelo, para ello  
dejamos reservada parte, el test, que no veria en el entrenamiento, para poder ver como predice.  
En este caso tenemos dos de las asignaturas mas importantes, **Acuracy** (punteria), o sea... del total de prediciones cuantas  
eran buenas, poco mas que decir y **loss**, que es como decir "como de malas son las prediciones que son incorrectas" dicho de otro modo, En un modelo que diferenciara aniamales, si se predice "Gorila" y la foto era un chimpancé la prediccion, sin ser correcta, no es lo peor del mundo, en cambio si predigera "berberecho" estaría muy mal.

Entrenamos el modelo con los datos de entrenamiento y luego lo evaluamos para ver si puede predecir emociones en datos que no ha visto.  
Segun chatgtp (y cito textualmente) "¡Es como enseñar a una computadora a leer expresiones faciales! 😊"


Es importante tener en cuenta que la salida del modelo no son strings, no nos dice directamente la emoción, nos da un numero entero, 
por lo que usar un diccionario se hace neecsario, al menos conocerlos: 
  
- 0 para 'Enfado' 😡
- 1 para 'Asco' 🤢
- 2 para 'Miedo' 😨
- 3 para 'Felicidad' 😄
- 4 para 'Tristeza' 😢
- 5 para 'Sorpresa' 😮
- 6 para 'Neutral' 😐



<img src="Imagenes/VGG16.jpg" alt="Imagen creada con inteligencia artificial y editada con Microsoft Paint" style="border-radius: 15px; width: 95%;">



*Imagen creada con inteligencia artificial*

## **DEFINICIÓN DEL MODELO BASE**  
>**aquí se define el modelo preentrenado VGG-16, utilizando pesos preentrenados de 'imagenet'**   
**y configurando la entrada para imágenes de 48x48 píxeles con 3 canales de color.**

In [15]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

## **AÑADIDO DE CAPAS**  
>**Con estas líneas adaptamos el modelo preentrenado VGG16 para que pueda hacer clasificaciones de**    
**expresiones faciales en nuestro caso, teniendo como entrada imágenes de 48x48 en escala de grises.**

In [16]:
model_vgg = models.Sequential()
model_vgg.add(layers.Conv2D(3, (1, 1), input_shape=(48, 48, 1)))  # Convertimos a 3 canales (o sea, colores... guiño guiño)
model_vgg.add(base_model)
model_vgg.add(layers.Flatten())
model_vgg.add(layers.Dense(4096, activation='relu'))
model_vgg.add(layers.Dense(4096, activation='relu'))
model_vgg.add(layers.Dense(7, activation='softmax'))  # 7 expresiones faciales.

C:\Users\pepea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## **CONGELADO DE CAPAS**  
>**Congelar las capas del modelo base (VGG-16) significa mantener sus pesos preentrenados,**    
**lo que aprovecha el conocimiento aprendido en imágenes generales. Esto acelera el entrenamient**  
**y reduce el riesgo de sobreajuste al adaptar el modelo para clasificar expresiones faciales en**    
**imágenes de 48x48 píxeles en escala de grises.**

In [17]:
for layer in base_model.layers:
    layer.trainable = False

## **COMPILADO DEL MODELO VGG**

In [18]:
model_vgg.compile(optimizer=optimizers.Adam(learning_rate=0.0005),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

## **ENTRENAMIENTO DEL MODELO VGG**

In [19]:
history = model_vgg.fit(X_train, y_train, epochs=25, batch_size=65, 
                        validation_data=(X_val, y_val), verbose=1)

Epoch 1/25
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 1298s 836ms/step - accuracy: 0.3686 - loss: 1.6483 - val_accuracy: 0.4880 - val_loss: 1.3291
Epoch 2/25
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 1294s 835ms/step - accuracy: 0.5264 - loss: 1.2363 - val_accuracy: 0.5594 - val_loss: 1.1594
Epoch 3/25
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 1295s 836ms/step - accuracy: 0.6132 - loss: 1.0189 - val_accuracy: 0.6048 - val_loss: 1.0534
Epoch 4/25
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 1295s 836ms/step - accuracy: 0.7059 - loss: 0.7962 - val_accuracy: 0.6714 - val_loss: 0.8967
Epoch 5/25
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 1294s 836ms/step - accuracy: 0.7921 - loss: 0.5774 - val_accuracy: 0.7242 - val_loss: 0.7895
Epoch 6/25
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 1295s 836ms/step - accuracy: 0.8613 - loss: 0.3987 - val_accuracy: 0.7492 - val_loss: 0.7395
Epoch 7/25
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 1293s 834ms/step - accuracy: 0.9116 - loss: 0.2671 - val_accuracy: 0.7778 - val_loss: 0.6931
Epoch 8/25
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 1293s 835ms/s

8 HORAS Y 30 MINUTOS DE ENTRENAMIENTO

## **EVALUACION CONTRA EL TEST** 
>**El modelo, en su entrenamiento, no ha visto estos ni de refilon (si, otra vez)**

In [24]:
loss, accuracy = model_vgg.evaluate(X_test, y_test, verbose=0)
print(f'Loss en el conjunto de prueba: {loss:.4f}')
print(f'Accuracy en el conjunto de prueba: {accuracy*100:.2f}%')

Loss en el conjunto de prueba: 0.7857
Accuracy en el conjunto de prueba: 84.41%


Loss en el conjunto de prueba: 0.7857  
Accuracy en el conjunto de prueba: 84.41%  

## **PREDICCIONES VGG**

In [21]:
y_pred_vgg = model_cnn.predict(X_test)
y_pred_classes_vgg = np.argmax(y_pred_vgg, axis=1)

394/394 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step


## **CLASSIFICATION REPORT CNN**

In [22]:
class_names = ['Enfado', 'Asco', 'Miedo', 'Felicidad', 'Tristeza', 'Sorpresa', 'Neutral']
print(classification_report(y_test, y_pred_classes_vgg, target_names=class_names))

              precision    recall  f1-score   support

      Enfado       0.80      0.73      0.76      1797
        Asco       0.98      0.99      0.98      1798
       Miedo       0.71      0.71      0.71      1798
   Felicidad       0.86      0.88      0.87      1798
    Tristeza       0.70      0.71      0.71      1798
    Sorpresa       0.90      0.91      0.90      1798
     Neutral       0.73      0.76      0.74      1798

    accuracy                           0.81     12585
   macro avg       0.81      0.81      0.81     12585
weighted avg       0.81      0.81      0.81     12585



## **GUARDADO DEL MODELO VGG**

In [23]:
model_vgg.save(os.path.join(save_dir, 'modelo_vgg.h5'))

save_model(model_vgg, os.path.join(save_dir, 'modelo_vgg.keras'))

## **RESUMEN/EXPLICACIÓN DEL MODELO VGG16**

Este modelo VGG16 modificado está diseñado para reconocer expresiones faciales en fotografías en blanco y negro de 48x48 píxeles, aunque es necesario convertirlas a 3 canales (RGB) para poder utilizar el modelo base VGG16 preentrenado en ImageNet.

> 🤫 **Nota**: "RGB" significa "Red, Green, Blue" en inglés, que son los colores primarios usados en pantallas para mostrar colores.

### **Capas Convolucionales (VGG16):**

Utilizaremos el **modelo VGG16** como base, el cual ha sido previamente entrenado con un gran conjunto de datos generales.  
Aprovecharemos los pesos y conocimientos adquiridos durante su entrenamiento, que consiste en múltiples capas convolucionales  
y de agrupación para detectar características complejas como bordes, texturas y patrones en las imágenes.  

### **Capas Densas Personalizadas:**

Se añaden capas densas para interpretar las características extraídas por VGG16 y clasificar las imágenes en una de las 7 expresiones faciales.    
Estas capas densas actúan como "inspectores" que determinan qué emoción se observa en la imagen.  

### **Congelación de Capas:**

Todas las capas de VGG16 se congelan para mantener los pesos preentrenados obtenidos de ImageNet. Esto aprovecha el conocimiento previo      
del modelo base y evita que se sobrescriban durante el entrenamiento con nuevos datos.  

### **Compilación del Modelo VGG16:**

El modelo se compila utilizando el optimizador Adam con una tasa de aprendizaje de 0.0005 y la función de pérdida **'sparse_categorical_crossentropy'**.    
Esto prepara al modelo para el entrenamiento, especificando cómo debe actualizarse durante el proceso de aprendizaje.  

> **Compilar** en el contexto de la programación y el aprendizaje automático significa preparar un programa o modelo para su ejecución o entrenamiento,  
optimizando el código fuente y las operaciones para la máquina.  
>
> **'sparse_categorical_crossentropy'** es una función de pérdida utilizada en problemas de clasificación donde las etiquetas son números enteros (sparse),  
es decir, no están codificadas en formato one-hot. Calcula la discrepancia entre las etiquetas reales y las predicciones del modelo para mejorar su  
capacidad de clasificación.

### **Entrenamiento del Modelo:**

El modelo se entrena durante 25 épocas utilizando datos de entrenamiento con lotes de 65 imágenes cada uno. Se utiliza un conjunto de validación  
para monitorear el rendimiento y evitar el sobreajuste.  

### **Evaluación del Modelo:**

Después del entrenamiento, se evalúa el modelo utilizando un conjunto de prueba reservado previamente. Se calcula la precisión (accuracy) y otras  
métricas para comprender qué tan bien generaliza el modelo con nuevos datos.  

### **Guardado del Modelo:**

Hacemos lo mismo que con el modelo anterior, obviamente cambiando los nombres.  

En este caso, la salida del modelo también es numérica, por lo que vuelve a ser necesario un diccionario para interpretar las predicciones. El diccionario es el mismo.
- 0 para 'Enfado' 😡
- 1 para 'Asco' 🤢
- 2 para 'Miedo' 😨
- 3 para 'Felicidad' 😄
- 4 para 'Tristeza' 😢
- 5 para 'Sorpresa' 😮
- 6 para 'Neutral' 😐

**ChatGPT dice**: ¡El modelo VGG16 es una poderosa herramienta para el reconocimiento de expresiones faciales, adaptada y optimizada para este desafío específico!  

## **¿POR QUÉ USAR DOS MODELOS?** 

Reconocer expresiones faciales es una tarea relativamente simple para los seres humanos, pero para los modelos de aprendizaje automático no es tan fácil. Interpretar y distinguir diferencias sutiles entre expresiones emocionales puede ser complicado para un solo modelo. Por lo tanto, para aumentar la confianza en las predicciones, se han entrenado dos modelos.

Ambos modelos están entrenados con el mismo conjunto de datos (Fer2013, acuerdate) y son bastante similares en su arquitectura. Sin embargo, el programa realiza predicciones con ambos modelos y selecciona la predicción que tenga el mayor nivel de confianza. En casos donde ambos modelos predicen diferentes emociones pero con igual nivel de confianza, se requerirá una revisión humana para tomar la decisión final.

Para mitigar el sesgo y sobreajuste debido a la utilización del mismo conjunto de datos, ambos modelos serán reentrenados con un nuevo dataset. Esto permitirá que los modelos generalicen mejor las predicciones a nuevas imágenes y situaciones.

Este enfoque de utilizar dos modelos y validar las predicciones con criterios de confianza y revisión humana ayuda a mejorar la precisión y fiabilidad del sistema de reconocimiento de expresiones faciales.